### Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
import pathlib
from tqdm import tqdm
import random
import pandas as pd
import os, sys, inspect
import torchvision as tv
import torchvision.models as models
import argparse
import time
from scipy.stats import binom
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import pickle as pkl
from scipy.optimize import brentq
from scipy.stats import binom, beta
from scipy.special import softmax
import pdb
import glob
import seaborn as sns
dirname = str(pathlib.Path().absolute())
sys.path.insert(1, os.path.join(sys.path[0], '../'))
from matplotlib.font_manager import FontProperties


### Helping function for PCOSQ

In [2]:
def NoisyRC(range_bounds, D, sigma):
    """
    Noisy Range Count for float values with Gaussian noise.

    Parameters:
    range_bounds (tuple): A tuple (a, b) representing the range [a, b].
    D (list): The sorted dataset.
    sigma (float): The standard deviation of the Gaussian noise.

    Returns:
    int: The noisy count of elements in the range [a, b].
    """
    a, b = range_bounds
    count = sum(1 for z in D if a <= z <= b)
    noise = np.random.normal(0, sigma)
    noisy_count = count + noise
    return max(0, int(np.floor(noisy_count)))  # Ensure non-negative count

def PrivQuant(D, alpha, rho, lower_bound=0, upper_bound=1, delta=1e-10):
    """
    Differentially Private Quantile Approximation Algorithm without integer conversion.

    Parameters:
    D (list): The sorted dataset.
    alpha (float): The quantile level (e.g., 0.5 for median).
    rho (float): The privacy parameter (smaller = more private).
    lower_bound (float): Lower bound of the search space.
    upper_bound (float): Upper bound of the search space.
    delta (float): Small positive value to ensure convergence.

    Returns:
    float: A differentially private approximation of the quantile x_{(m)}.
    """

    
    n = len(D)
    max_iterations = int(np.ceil(np.log2((upper_bound - lower_bound) / delta)))
    sigma = np.sqrt(max_iterations / (2 * rho))  # Noise scale for Gaussian mechanism
    m = int(np.ceil((1 - alpha) * (n + 1)))

    left, right = lower_bound, upper_bound

    for i in range(max_iterations):
        mid = (left + right) / 2
        c = NoisyRC((lower_bound, mid), D, sigma)
        
        if c < m:
            left = mid + delta
        else:
            right = mid

    return np.round((left + right) / 2, 2)

### Helping function for EXPONQ

In [3]:

def get_qtilde(n,alpha,gamma,epsilon,m):
    qtilde = (n+1)*(1-alpha)/(n*(1-gamma*alpha))+2/(epsilon*n)*np.log(m/(gamma*alpha))
    qtilde = min(qtilde, 1-1e-12)
    return qtilde

def generate_scores(n):
    return np.random.uniform(size=(n,))

def hist_2_cdf(cumsum, bins, n):
    def _cdf(t):
        if t > bins[-2]:
            return 1.0
        elif t < bins[1]:
            return 0.0
        else:
            return 1-cumsum[np.searchsorted(bins, t)]/n
    return _cdf




    



def get_private_quantile(scores, alpha, epsilon, gamma, bins):
    n = scores.shape[0]
    epsilon_normed = epsilon*min(alpha, 1-alpha)
    # Get the quantile
    qtilde = get_qtilde(n, alpha, gamma, epsilon, bins.shape[0])
    scores = scores.squeeze()
    score_to_bin = np.digitize(scores,bins)
    binned_scores = bins[np.minimum(score_to_bin,bins.shape[0]-1)]
    w1 = np.digitize(binned_scores, bins)
    w2 = np.digitize(binned_scores, bins, right=True)
    # Clip bins
    w1 = np.maximum(np.minimum(w1,bins.shape[0]-1),0)
    w2 = np.maximum(np.minimum(w2,bins.shape[0]-1),0)
    lower_mass = np.bincount(w1,minlength=bins.shape[0]).cumsum()/qtilde
    upper_mass = (n-np.bincount(w2,minlength=bins.shape[0]).cumsum())/(1-qtilde)
    w = np.maximum( lower_mass , upper_mass )
    sampling_probabilities = softmax(-(epsilon_normed/2)*w)
    # Check
    sampling_probabilities = sampling_probabilities/sampling_probabilities.sum()
    qhat = np.random.choice(bins,p=sampling_probabilities)
    return qhat

# Optimal gamma is a root.
def get_optimal_gamma(scores,n,alpha,m,epsilon):
    a = alpha**2
    b = - ( alpha*epsilon*(n+1)*(1-alpha)/2 + 2*alpha )
    c = 1
    best_q = 1
    gamma1 = (-b + np.sqrt(b**2 - 4*a*c))/(2*a)
    gamma2 = (-b - np.sqrt(b**2 - 4*a*c))/(2*a)

    gamma1 = min(max(gamma1,1e-12),1-1e-12)
    gamma2 = min(max(gamma2,1e-12),1-1e-12)

    bins = np.linspace(0,1,m)

    q1 = get_private_quantile(scores, alpha, epsilon, gamma1, bins)
    q2 = get_private_quantile(scores, alpha, epsilon, gamma2, bins)

    return (gamma1, q1) if q1 < q2 else (gamma2, q2)

def get_optimal_gamma_m(n, alpha, epsilon):
    candidates_m = np.logspace(4,6,50).astype(int)
    scores = np.random.rand(n,1)
    best_m = int(1/alpha)
    best_gamma = 1
    best_q = 1
    for m in candidates_m:
        gamma, q = get_optimal_gamma(scores,n,alpha,m,epsilon)
        if q < best_q:
            best_q = q
            best_m = m
            best_gamma = gamma
    return best_m, best_gamma




def get_conformal_scores(scores, labels):
    conformal_scores = torch.tensor([scores[i,labels[i]] for i in range(scores.shape[0])]) 
    return conformal_scores 

def get_shat_from_scores_private(scores, alpha, epsilon, gamma, score_bins):
    shat = get_private_quantile(scores, alpha, epsilon, gamma, score_bins)
    return shat

### Helping function for model training

In [4]:

def get_model(modelname):
    """
    Load a pre-trained model by name.

    Args:
        modelname (str): Name of the model (e.g., 'ResNet152').

    Returns:
        torch.nn.Module: Pre-trained model.
    """
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    
    # Use the new weights parameter
    if modelname == 'ResNet18':
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    elif modelname == 'ResNet50':
        model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    elif modelname == 'ResNet101':
        model = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V1)
    elif modelname == 'ResNet152':
        model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V1)
    elif modelname == 'ResNeXt101':
        model = models.resnext101_32x8d(weights=models.ResNeXt101_32X8D_Weights.IMAGENET1K_V1)
    elif modelname == 'VGG16':
        model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    elif modelname == 'ShuffleNet':
        model = models.shufflenet_v2_x1_0(weights=models.ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1)
    elif modelname == 'Inception':
        model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)
    elif modelname == 'DenseNet161':
        model = models.densenet161(weights=models.DenseNet161_Weights.IMAGENET1K_V1)
    else:
        raise NotImplementedError(f"Model {modelname} not supported")

    model.eval()
    model.to(device)
    return model

# Computes logits and targets from a model and loader
def get_logits_targets(model, loader):
    """
    Compute logits and targets for a dataset using a model.

    Args:
        model (torch.nn.Module): Pre-trained model.
        loader (torch.utils.data.DataLoader): DataLoader for the dataset.

    Returns:
        torch.Tensor: Tuple of (logits, targets).
    """
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    logits = torch.zeros((len(loader.dataset), 1000))  # 1000 classes in ImageNet
    labels = torch.zeros((len(loader.dataset),))
    i = 0

    print(f'Computing logits for model (only happens once).')
    with torch.no_grad():
        for x, targets in tqdm(loader, desc="Processing batches"):
            batch_logits = model(x.to(device)).detach().cpu()
            logits[i:(i + x.shape[0]), :] = batch_logits
            labels[i:(i + x.shape[0])] = targets.cpu()
            i += x.shape[0]

    # Construct the dataset
    dataset_logits = torch.utils.data.TensorDataset(logits, labels.long())
    return dataset_logits



def get_logits_dataset(modelname, datasetname, datasetpath, cache= dirname + '/.cache/'):
    fname = cache + datasetname + '/' + modelname + '.pkl' 

    # If the file exists, load and return it.
    if os.path.exists(fname):
        with open(fname, 'rb') as handle:
            return pkl.load(handle)

    # Else we will load our model, run it on the dataset, and save/return the output.
    model = get_model(modelname)

    transform = transforms.Compose([
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std =[0.229, 0.224, 0.225])
                    ])
    
    dataset = torchvision.datasets.ImageNet(datasetpath, split='val', transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, pin_memory=True, num_workers=4)

    # Get the logits and targets
    dataset_logits = get_logits_targets(model, loader)

    # Save the dataset 
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    with open(fname, 'wb') as handle:
        pkl.dump(dataset_logits, handle, protocol=pkl.HIGHEST_PROTOCOL)

    return dataset_logits

    
def fix_randomness(seed=0):
    np.random.seed(seed=seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)

def get_imagenet_classes():
    df = pd.read_csv(dirname + '/map_clsloc.txt', delimiter=' ')
    arr = df['name'].to_numpy()
    return arr

def get_metrics_precomputed(est_labels,labels,losses,num_classes):
    labels = torch.nn.functional.one_hot(labels,num_classes)
    empirical_losses = (losses.view(1,-1) * (labels * (1-est_labels))).sum(dim=1)
    sizes = est_labels.sum(dim=1)
    return empirical_losses, sizes 


def platt_logits(calib_dataset, max_iters=10, lr=0.01, epsilon=0.01):
    calib_loader = torch.utils.data.DataLoader(calib_dataset, batch_size=1024, shuffle=False, pin_memory=True) 
    nll_criterion = nn.CrossEntropyLoss().cuda()

    T = nn.Parameter(torch.Tensor([1.3]).cuda())

    optimizer = optim.SGD([T], lr=lr)
    for iter in range(max_iters):
        T_old = T.item()
        for x, targets in calib_loader:
            optimizer.zero_grad()
            x = x.cuda()
            x.requires_grad = True
            out = x/T
            loss = nll_criterion(out, targets.long().cuda())
            loss.backward()
            optimizer.step()
        if abs(T_old - T.item()) < epsilon:
            break
    return T 


### Helping function for experiment

In [5]:
def trial_precomputed(conformal_scores, raw_scores, alpha, epsilon, num_calib,score_bins,gamma):
        
    total=conformal_scores.shape[0]
    perm = torch.randperm(conformal_scores.shape[0])
    conformal_scores = conformal_scores[perm]
    raw_scores = raw_scores[perm]
    calib_conformal_scores, val_conformal_scores = (1-conformal_scores[0:num_calib], 1-conformal_scores[num_calib:])
    calib_raw_scores, val_raw_scores = (1-raw_scores[0:num_calib], 1-raw_scores[num_calib:])

    shat = get_shat_from_scores_private(calib_conformal_scores, alpha, epsilon, gamma, score_bins)
    threshold_PrivQuant = PrivQuant(calib_conformal_scores, alpha, rho=epsilon, lower_bound=0, upper_bound=1, delta = 1e-16) 
   

    corrects = (val_conformal_scores) < shat
    corrects_PrivQuant = (val_conformal_scores) < threshold_PrivQuant
    sizes = ((val_raw_scores) < shat).sum(dim=1)
    sizes_PrivQuant = ((val_raw_scores) < threshold_PrivQuant).sum(dim=1)

    return corrects.float().mean().item(),corrects_PrivQuant.float().mean().item(),torch.tensor(sizes),torch.tensor(sizes_PrivQuant), shat,threshold_PrivQuant



### Experiment

In [6]:
def experiment(alpha, epsilons, num_calib, batch_size, imagenet_val_dir):
    # Clear cache files
    cache_files = glob.glob(f'.cache/opt_{alpha}_*_dataframe_trial.pkl')
    for cache_file in cache_files:
        os.remove(cache_file)
    
    df_list = []
    for epsilon in epsilons:
        # Create the .cache directory if it doesn't exist
        os.makedirs('.cache', exist_ok=True)
        
        fname = f'.cache/opt_{alpha}_{epsilon}_{num_calib}_dataframe_trial.pkl'
        mstar, gammastar = get_optimal_gamma_m(num_calib, alpha, epsilon)
        m = mstar
        gamma = gammastar
        score_bins = np.linspace(0, 1, m)
    
        # Define the expected columns
        expected_columns = ["$\\hat{s}$", "$\\hat{q}_$PCOQS", "EXPONQ", "PCOQS", "sizes_EXPONQ", "sizes_PCOQS", "$\\alpha$", "$\\epsilon$"]
    
        try:
            # Try to load the cached DataFrame
            df = pd.read_pickle(fname)
        except FileNotFoundError:
            # If the cached file doesn't exist, compute the DataFrame
            dataset_precomputed = get_logits_dataset('ResNet152', 'Imagenet', imagenet_val_dir)
            print('Dataset loaded')
        
            classes_array = get_imagenet_classes()
            T = platt_logits(dataset_precomputed)
        
            logits, labels = dataset_precomputed.tensors
            scores = (logits / T.cpu()).softmax(dim=1)
        
            with torch.no_grad():
                conformal_scores = get_conformal_scores(scores, labels)
                local_df_list = []
                for i in tqdm(range(num_trials)):
                    cvg1, cvg2, szs1, szs2, shat, threshold_PrivQuant = trial_precomputed(conformal_scores, scores, alpha, epsilon, num_calib,score_bins,gamma)
                    dict_local = {
                        "$\\hat{s}$": shat,
                        "$\\hat{q}_$PCOQS": threshold_PrivQuant,
                        "EXPONQ": cvg1,
                        "PCOQS": cvg2,
                        "sizes_EXPONQ": [szs1],
                        "sizes_PCOQS": [szs2],
                        "$\\alpha$": alpha,
                        "$\\epsilon$": epsilon
                    }
                    df_local = pd.DataFrame(dict_local)
                    local_df_list.append(df_local)
        
                # Combine all local DataFrames into one
                df = pd.concat(local_df_list, axis=0, ignore_index=True)
        
                # Ensure the DataFrame has the expected columns
                df = df.reindex(columns=expected_columns)
        
                
            df_list.append(df)


    return df_list


if __name__ == "__main__":
    sns.set(palette='pastel',font='serif')
    sns.set_style('white')
    fix_randomness(seed=0)

    imagenet_val_dir = '/home/ogonna/Differential_Privacy_Project/Conformal_Prediction_Project/ImageNet_Data_Simulation/' # TODO: put your imagenet directory here

    alpha = 0.1
    epsilons = [0.05,0.1,0.5,1,3,5]
    num_calib = 30000 
    num_trials = 100

    results=experiment(alpha, epsilons, num_calib, batch_size=128, imagenet_val_dir=imagenet_val_dir)

Dataset loaded


  0%|                                                     | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_221547/1696106157.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return corrects.float().mean().item(),corrects_PrivQuant.float().mean().item(),torch.tensor(sizes),torch.tensor(sizes_PrivQuant), shat,threshold_PrivQuant
100%|█████████████████████████████████████████████| 2/2 [00:25<00:00, 12.87s/it]


Dataset loaded


  0%|                                                     | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_221547/1696106157.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return corrects.float().mean().item(),corrects_PrivQuant.float().mean().item(),torch.tensor(sizes),torch.tensor(sizes_PrivQuant), shat,threshold_PrivQuant
100%|█████████████████████████████████████████████| 2/2 [00:25<00:00, 12.90s/it]


Dataset loaded


...

  0%|                                                     | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_221547/1696106157.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return corrects.float().mean().item(),corrects_PrivQuant.float().mean().item(),torch.tensor(sizes),torch.tensor(sizes_PrivQuant), shat,threshold_PrivQuant
100%|█████████████████████████████████████████████| 2/2 [00:25<00:00, 12.96s/it]


Dataset loaded


  0%|                                                     | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_221547/1696106157.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return corrects.float().mean().item(),corrects_PrivQuant.float().mean().item(),torch.tensor(sizes),torch.tensor(sizes_PrivQuant), shat,threshold_PrivQuant
100%|█████████████████████████████████████████████| 2/2 [00:25<00:00, 12.99s/it]


Dataset loaded


  0%|                                                     | 0/2 [00:00<?, ?it/s]/tmp/ipykernel_221547/1696106157.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return corrects.float().mean().item(),corrects_PrivQuant.float().mean().item(),torch.tensor(sizes),torch.tensor(sizes_PrivQuant), shat,threshold_PrivQuant
100%|█████████████████████████████████████████████| 2/2 [00:26<00:00, 13.01s/it]


### Saving the results

In [7]:
save_path = 'df_list_ImageNet_results_trial.pkl'
with open(save_path, 'wb') as f:
            pkl.dump(results, f)


### Processing and saving result for plotting

In [8]:
def compute_trial_averages(size_series, trials=100, eval_points=20000):
    """
    Compute average set size per trial from size series
    Returns: Array of 100 average sizes (one per trial)
    """
    # First flatten and convert all values to floats
    sizes = []
    for val in size_series.explode().dropna():
        if isinstance(val, torch.Tensor):
            sizes.append(float(val.item()))
        else:
            sizes.append(float(val))
    
    if len(sizes) != trials * eval_points:
        print(f"Warning: Expected {trials*eval_points} size points, got {len(sizes)}")
        return np.array([])
    
    # Reshape to (trials, eval_points) and compute trial averages
    size_array = np.array(sizes).reshape(trials, eval_points)
    return np.mean(size_array, axis=1)

def safe_to_dataframe(data_dict):
    """Convert dictionary to DataFrame, handling unequal lengths"""
    if not data_dict:
        return pd.DataFrame()
    
    max_len = max(len(v) for v in data_dict.values())
    padded = {k: np.pad(v, (0, max_len - len(v)), 
             mode='constant', constant_values=np.nan)
             for k, v in data_dict.items()}
    return pd.DataFrame(padded)

def main():
    # Load your data
    try:
        with open('df_list_ImageNet_results_trial.pkl', 'rb') as f:
            df_list = pkl.load(f)
    except FileNotFoundError:
        print("Error: Input file not found")
        return
    except pkl.PickleError:
        print("Error: Could not unpickle the file")
        return

    # Epsilon values
    epsilons = [0.05, 0.1, 0.5, 1, 3, 5]

    # Initialize storage
    results = {
        'coverage': {epsilon: {} for epsilon in epsilons},
        'avg_size': {epsilon: {} for epsilon in epsilons}  
    }

    # Process each epsilon's DataFrame
    for epsilon, df in zip(epsilons, df_list):
        try:
            # Coverage data
            for method in ["EXPONQ", "PCOQS"]:
                if method in df.columns:
                    cov_data = df[method].dropna().astype(float).values
                    results['coverage'][epsilon][method] = cov_data
            
            # Size data - compute trial averages
            for method in ["EXPONQ", "PCOQS"]:
                size_key = f"sizes_{method}"
                if size_key in df.columns:
                    trial_avgs = compute_trial_averages(df[size_key])
                    if len(trial_avgs) > 0:
                        results['avg_size'][epsilon][method] = trial_avgs
                    else:
                        print(f"Warning: No size averages for {method} at epsilon={epsilon}")
        except Exception as e:
            print(f"Error processing epsilon={epsilon}: {str(e)}")
            continue

    # Save coverage data
    for ε in epsilons:
        if results['coverage'][epsilon]:
            df = safe_to_dataframe(results['coverage'][epsilon])
            if not df.empty:
                df.to_csv(f'coverage_epsilon_{epsilon}.csv', index=False)

    # Save average size data (one file per epsilon)
    for ε in epsilons:
        if results['avg_size'][epsilon]:
            df = safe_to_dataframe(results['avg_size'][epsilon])
            if not df.empty:
                df.to_csv(f'avg_size_epsilon_{epsilon}.csv', index=False)
            else:
                print(f"No average size data for epsilon={epsilon}")

    print("Processing complete. Files saved:")
    print("- coverage_epsilon_[epsilon].csv")
    print("- avg_size_epsilon_[epsilon].csv")

if __name__ == "__main__":
    main()

Processing complete. Files saved:
- coverage_epsilon_[epsilon].csv
- avg_size_epsilon_[epsilon].csv
